In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\anaconda3\envs\ocr2\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Ramon Kramer Shin\AppData\Local\Temp\ipykernel_19668\3742080765.py", line 1, in <module>
    import tensorflow as tf
  File "D:\anaconda3\envs\ocr2\lib\site-packages\tensorflow\__init__.py", line 41, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "D:\anaconda3\envs\ocr2\lib\site-packages\tensorflow\python\__init__.py", line 46, in <module>
    from tensorflow.python import data
  File "D:\anaconda3\envs\ocr2\lib\site-packages\tensorflow\python\data\__init__.py", line 25, in <module>
    from tensorflow.python.data import experimental
  File "D:\anaconda3\envs\ocr2\lib\site-packages\tensorflow\python\data\experimental\__init__.py", line 97, in <module>
    from tensorflow.python.data.experimental import service
  File "D:\anaconda3\envs\o

TypeError: object of type 'NoneType' has no len()

In [ ]:
# GPU 강제 사용
import os
from tensorflow.python.platform import build_info as tf_build_info

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
print(tf_build_info.build_info['cuda_version'])
# print(tf_build_info.build_info['cudnn_version_number'])
tf.config.list_physical_devices('GPU')
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#     except RuntimeError as e:
#         # Memory growth must be set before GPUs have been initialized
#         print(e)

In [ ]:
from tensorflow.keras.datasets.mnist import load_data
# 1) 데이터 셋 : tf.keras.datasets.mnist, load_data()

(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
# 2) 훈련 데이터 셋과 테스트 데이터 셋을 분할 (테스트 데이터 세트 20%)
# 해당 데이터는 60000:10000으로 이미 나뉘어져 있었음.
# 실제 데이터 활용 시 train_test_split(x, y, stratify=y, test_size=0.2) 적용
# from sklearn.model_selection import train_test_split

# 3) 훈련 데이터 셋은 훈련, 검증으로 분할 (분할 율 : 8:2)
# x_train, x_val, y_train, y_val = train_test_split(x, y, stratify=y, test_size=0.2)

In [ ]:
# 이미지 프리뷰
plt.imshow(x_train[0], cmap='binary')
print(y_train.shape)

In [ ]:
# 개별 데이터 갯수 카운트
print(np.bincount(y_train), np.bincount(y_test))

In [ ]:
# 원 핫 인코딩 과정
from tensorflow.keras.utils import to_categorical


def one_hot_encoding(x_, y_):
    # 데이터 정규화 과정
    x_ = x_ / 255  # 0~255 까지의 값을 0~1로 출력
    x_ = x_.reshape(-1, 28, 28, 1)
    y_encoded = to_categorical(y_)  # 데이터 이진화
    return x_, y_encoded


x_train, y_train_encoded = one_hot_encoding(x_train, y_train)
x_test, y_test_encoded = one_hot_encoding(x_test, y_test)

print(x_train.shape, y_train_encoded.shape , '\n',
      x_test.shape, y_test_encoded.shape)

In [ ]:
# 분류 알고리즘 불러오기
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, MaxPool2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential([
    Conv2D(kernel_size=(3, 3), filters=64, input_shape=(28, 28, 1), padding='same', activation='relu'),
    Conv2D(kernel_size=(3, 3), filters=64, padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),

    Conv2D(kernel_size=(3, 3), filters=128, padding='same', activation='relu'),
    Conv2D(kernel_size=(3, 3), filters=256, padding='valid', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),

    Flatten(),
    Dense(units=512, activation='relu'),
    Dropout(0.5),
    Dense(units=256, activation='relu'),
    Dropout(0.5),
    Dense(units=10, activation='softmax'),

])

In [ ]:
from tensorflow.keras.metrics import CategoricalCrossentropy

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_train, y_train_encoded, batch_size=100, epochs=10, validation_data=(x_test, y_test_encoded))

In [ ]:
result = model.evaluate(x_test, y_test_encoded)
print("최종 예측 성공률(%): ", result[1]*100)

In [ ]:
# 학습한 모델 저장
# model.save_weights('./model/model.ckpt')
model.save('./model/digit_classifier.h5')
model.save('./def_model/classifier')

In [ ]:
plt.imsave('d',)